In [32]:
import pandas as pd

df = pd.read_csv('./data/data2023.csv')

In [33]:
df.dropna(inplace=True)
df['smpUsd'] = df['smp'] / df['exchangeRate']
df['idmUsd'] = df['idm'] / df['exchangeRate']

In [34]:
NEGATIVE_IMBALANCE_PENALTY = 1.03
POSITIVE_IMBALANCE_PENALTY = 0.97


def calculate_negative_imbalance_price(smp, mcp):
    return max(smp, mcp) * NEGATIVE_IMBALANCE_PENALTY


def calculate_positive_imbalance_price(smp, mcp):
    return min(smp, mcp) * POSITIVE_IMBALANCE_PENALTY

In [35]:
df["positive_imbalance_price"] = df.apply(
    lambda row: calculate_positive_imbalance_price(row["smpUsd"], row["mcpUsd"]), axis=1
)
df["negative_imbalance_price"] = df.apply(
    lambda row: calculate_negative_imbalance_price(row["smpUsd"], row["mcpUsd"]), axis=1
)

In [36]:
def calculate_shortselling_profit(idm, negative_imbalance_price):
    return max(idm - negative_imbalance_price, 0)


def calculate_long_profit(idm, positive_imbalance_price):
    return max(positive_imbalance_price - idm, 0)

In [37]:
def calculate_shortselling_loss(idm, negative_imbalance_price):
    return max(negative_imbalance_price - idm, 0)

def calculate_long_loss(idm, positive_imbalance_price):
    return max(idm - positive_imbalance_price, 0)

In [38]:
df["shortselling_profit"] = df.apply(
    lambda row: calculate_shortselling_profit(
        row["idmUsd"], row["negative_imbalance_price"]
    ),
    axis=1,
)
df["long_profit"] = df.apply(
    lambda row: calculate_long_profit(row["idmUsd"], row["positive_imbalance_price"]),
    axis=1,
)

df["total_profit"] = df["shortselling_profit"] + df["long_profit"]

In [39]:
df["shortselling_loss"] = df.apply(
    lambda row: calculate_shortselling_loss(
        row["idmUsd"], row["negative_imbalance_price"]
    ),
    axis=1,
)

df["long_loss"] = df.apply(
    lambda row: calculate_long_loss(row["idmUsd"], row["positive_imbalance_price"]),
    axis=1,
)

df["total_loss"] = df["shortselling_loss"] + df["long_loss"]

In [40]:
df['is_shortselling_profitable'] = df['shortselling_profit'] > 0
df['is_long_profitable'] = df['long_profit'] > 0

In [42]:
from autogluon.tabular import TabularDataset, TabularPredictor

predictor = TabularPredictor.load("AutogluonModels/HourAndDayMediumUsdWithoutVolume") # One later model

In [43]:
df['dayOfWeek'] = pd.to_datetime(df['date']).dt.dayofweek

In [44]:
X = df[['mcpUsd','damVolume', 'idmUsd', 'hour', 'dayOfWeek']]
y = df['is_long_profitable']

In [45]:
X = pd.get_dummies(X, columns=['hour', 'dayOfWeek'], drop_first=True)

In [46]:
data = TabularDataset(X.join(y))

In [47]:
y_pred = predictor.predict(data)
y_pred_proba = predictor.predict_proba(data)

In [48]:
predictor.evaluate(data)

{'f1': 0.19619865113427346,
 'accuracy': 0.8503082895638273,
 'balanced_accuracy': 0.5485624043085415,
 'mcc': 0.14879559681918125,
 'roc_auc': 0.776132023213939,
 'precision': 0.350109409190372,
 'recall': 0.1362862010221465}

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(data['is_long_profitable'], y_pred)

array([[7287,  297],
       [1014,  160]])

In [50]:
POSITIVE_IMBALANCE_PENALTY = 0.97
NEGATIVE_IMBALANCE_PENALTY = 1.03
def calculate_pnl(mcp, smp, idm, strategy):
    positive_imbalance_price = min(mcp, smp) * POSITIVE_IMBALANCE_PENALTY
    negative_imbalance_price = max(mcp, smp) * NEGATIVE_IMBALANCE_PENALTY
    if strategy == 'buy':
        return positive_imbalance_price - idm
    elif strategy == 'sell':
        return idm - negative_imbalance_price
    else:
        return 0


In [67]:
threshold = 0.8
m_y_pred = (y_pred_proba.iloc[:, 1] > threshold).astype(int)
df["strategy"] = m_y_pred.apply(lambda x: 'buy' if x == 1  else 'hold')
df["pnl"] = df.apply(
    lambda row: calculate_pnl(row["mcpUsd"], row["smpUsd"], row["idmUsd"], row["strategy"]),
    axis=1,
)

In [68]:
df['pnl'].sum()

-746.0000109309552

In [69]:
# 
confusion_matrix(data['is_long_profitable'], m_y_pred)

array([[7560,   24],
       [1166,    8]])